In [2]:
import re
import os
from glob import glob

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from operator import itemgetter
from langchain.schema import HumanMessage
from langchain_community.tools import TavilySearchResults
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from textwrap import dedent

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI Embeddings & LLM 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
chat_model = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


# 대화 메모리 설정
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# 벡터 스토어 로드 (이미 저장된 벡터 데이터 사용)
PERSIST_DIRECTORY = "vector_store/contents"  # 기존에 데이터 저장된 경로
COLLECTION_NAME = "contents"

vector_store = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model
)

# Retriever 설정 (유사한 웹툰 검색)
retriever = vector_store.as_retriever(search_type="mmr",search_kwargs={'k':10,'lambda_mult':0.25})


C:\Users\Playdata\AppData\Local\Temp\ipykernel_21008\3266707079.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21008\3266707079.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [ ]:
# db 검색 tool
@tool
def search_contents(query: str) -> list[Document]:
    """
    Vector Store에 저장된 웹툰 조회.
    """
    result = retriever.invoke(query)
    return result if result else [Document(page_content="검색 결과가 없습니다.")]


@tool
def classify_intent(user_query: str) -> str:
    """
    LLM을 사용하여 사용자의 의도, 감정, 말투를 분석하는 tool.
    """
    intent_prompt = f"""
    <basic role>
    사용자의 입력을 보고 의도와 감정, 말투를 분석하여 아래 중 하나로 분류하세요. 
    {{
    "가능한 의도": ["웹툰 추천 요청", "웹툰 정보 요청", "웹툰 인기 순위", "웹소설 추천 요청", "웹소설 정보 요청", "웹소설 인기 순위", "일반 대화", "인사"],
    "가능한 감정": ["평온", "기쁨", "슬픔", "화남", "기대", "장난"],
    "가능한 말투": ["반말", "존댓말"]
    }}
    </basic role>
    <rules>
    아니, 뭐해, 이딴, 아오 등은 사용자가 화났을 때 주로 사용.
    존댓말은 보통 "~요","~니다"로 끝남. 반말은 "~요","~니다"로 끝나지 않는 모든 말.
    {{
    examples of "존댓말": ["안녕하세요", "좋은 아침입니다", "확인했습니다", "금요일 웹툰 추천해주세요"],
    examples of "반말": ["안녕", "뭐해?", "어이없네", "금요일 웹툰 추천해줘"]
    }}
    "안녕"은 반말이야
    </rules>
    분석된 의도: , 분석된 감정: , 분석된 말투:

    사용자 입력: "{user_query}"
    """
    response = chat_model.invoke(intent_prompt)
    return response.content.strip()


@tool
def recommender(user_query: str) -> str:
    """
    a tool which recommends a list of webtoon(or webnovel) using LLM
    """
    recommend_prompt = f"""
    <role>
    recommend 5 webtoons via given (context) data
    
    사용자 입력: "{user_query}"

    
    """
    response = chat_model.invoke(recommend_prompt)
    return response.content.strip()

    

In [ ]:
# LLM 구성
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent("""
                   <role>
                    당신은 웹툰을 추천하는 챗봇입니다. 질문(question)을 분석하고 (context)에서 해당하는 웹툰을 찾아서 사용자에게 보여줍니다.
                    추천 전에 "건방지군. 감히 이 몸에게 질문을 하다니. 하지만 지금은 이런 몸이니...추천하마" 또는 "질문을 하는 사람은 잠깐 바보가 되지만, 질문하지 않는 자는 평생 바보로 살지. 너는 방금 바보를 벗어났다."와 같은 말을 하고 추천합니다.
                    사용자의 (user_query)에 따라 (context)에서 검색하여 알맞는 웹툰을 추천합니다.
                   </role>
                   <rules>
                    사용자가 특정 웹툰에 대한 정보를 물어보면 그 웹툰에 대한 정보를 자세히 알려주세요.
                    사용자가 특정 장르(genre)나 어떤 키워드(keywords)을 가진 웹툰을 추천해달라고 하면 그 장르나 키워드에 해당되는 5개 이상의 웹툰을 (context)에서 검색하여 반드시 아래 정보를 알려주세요:
                    작가(author):
                    장르(genre):
                    설명(description):
                    플랫폼(platform)
                    키워드(keywords):
                    사용자가 자극적인 웹툰을 찾으면 죽음, 복수, 19세 연령가 등 선정적인 웹툰을 추천해주세요.
                    사용자에게 웹툰을 추천할 때 장르나 키워드 데이터를 참고하세요.
                   </rules>
                    예시:
                        질문: "로판 웹툰을 추천해줘"
                        답변: 장르(genre)나 키워드(keyword) 안에서 로판이라는 단어를 찾아 추천
                    {context}
                    """
            ),
        ),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory()  # 메모리 설정
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# agent 구성
agent = create_tool_calling_agent(
    llm=model, tools=[search_contents, recommender, classify_intent], prompt=prompt_template
)
toolkit = [search_contents, recommender, classify_intent]
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)
runnable = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": itemgetter("question"),
        "history": itemgetter("history"),
    }
    | prompt_template
    | model
    | parser
)

chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=lambda session_id: memory.chat_memory,
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# 사용자 질문
query = "액션 웹툰 추천좀해줘"
context = 


In [9]:
# LLM 입력 메세지 구성
input_messages = [
    HumanMessage(content=f"사용자 질문: {query}"),
    HumanMessage(content=f"context: {context}")
]

In [10]:
# 응답 생성
response = agent_executor.invoke(
    {
        "question": query,
        "context": context,
        "history": history,
    }
)

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `recommender` with `{'user_query': '액션 웹툰'}`


다음은 액션 웹툰 추천 목록입니다:

1. **신의 탑 (Tower of God)** - 불확실한 세계에서 신의 탑을 오르는 소년의 이야기로, 다양한 캐릭터와 전투가 인상적인 작품입니다.

2. **머니게임 (Money Game)** - 생존을 위한 긴박한 게임과 그 속에서 벌어지는 인간의 본성과 심리를 다룬 액션 스릴러입니다.

3. **소녀의 세계 (A Girl's World)** - 청춘과 성장, 액션이 결합된 작품으로, 고등학생들의 다양한 일상과 갈등을 그려냅니다.

4. **레벨업 왕 (Level Up, Family)** - 게임과 현실이 뒤섞인 세계에서 주인공이 레벨업을 통해 강해져 가는 과정을 그린 액션 판타지입니다.

5. **더 킹 : 영원의 군주 (The King: Eternal Monarch)** - 평행 세계를 배경으로 한 액션과 로맨스가 어우러진 작품으로, 왕과 경찰의 협력이 중심 이야기입니다.

이 웹툰들은 액션 요소가 두드러지며, 흥미진진한 줄거리를 가지고 있어 추천드립니다!
Invoking: `search_contents` with `{'query': '액션'}`


[Document(metadata={'id': 65917920, 'platform': '카카오페이지', 'title': '홀아비와 오야마', 'type': '웹툰'}, page_content="id: 65917920, type: 웹툰, platform: 카카오페이지, title: 홀아비와 오야마, status: 완결, thumbnail: https://page-images.kakaoentcdn.com/download/resource?kid=c44QEP/hAFPLAjoLA/b5vCgtU0cAAoSG7Aa3RKL1&filename=o1, genre: 드라마, views: 91, rating: 10.0

tool 생성

In [ ]:
def webtoon_recommendation_prompt(user_query: str):
    '''웹툰 추천용 프롬프트 생성'''
    return ChatPromptTemplate.from_messages([
        ("system", """
        
        {context}"""),
        MessagesPlaceholder(variable_name="history"),
        ("human", user_query)
    ])

In [ ]:
  #      분석된 의도, 감정, 말투를 (response)에 포함하지 마세요.

In [ ]:
def recommend_webtoon(user_query: str):
    '''사용자 질문을 기반으로 웹툰 추천'''
    
    # 벡터 스토어에서 관련 웹툰 검색
    retrieved_docs = retriever.invoke(user_query)
    
    # 검색된 웹툰 정보 정리
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # 프롬프트 생성
    prompt = webtoon_recommendation_prompt(user_query)
    
    # 메모리에서 이전 대화 불러오기
    history_messages = memory.load_memory_variables({})["history"]

    # LLM에 추천 요청
    response = chat_model.invoke(prompt.format(history=history_messages, question=user_query, context=context))

    # 대화 저장 (히스토리 유지)
    memory.save_context({"question": user_query}, {"response": response.content})

    return response.content.strip()


In [ ]:
# 테스트 실행
query = "로판 웹툰 추천해줘"

In [ ]:
print(recommend_webtoon(query))

질문을 하는 사람이 잠깐 바보가 되지만, 질문하지 않는 자는 평생 바보로 살지. 너는 방금 바보를 벗어났다. 로판 웹툰을 추천해주겠다!

1. **파괴하러 왔습니다**
   - **작가**: 기쟈
   - **장르**: 로맨스 판타지
   - **설명**: 전능한 식물의 신으로 태어났지만, 격리실에 갇힌 200년의 삶을 뒤로 하고 후작 영애에게 빙의되어 복수를 다짐하는 이야기.
   - **플랫폼**: 카카오페이지
   - **키워드**: 로맨스판타지, 빙의물, 능력녀, 걸크러쉬, 정략결혼

2. **피폐 역하렘 남주들의 막내 처제가 되었다**
   - **작가**: 미르
   - **장르**: 로맨스 판타지
   - **설명**: 이 세계 뒤에서 힘을 쥐고 있는 남자들의 사랑을 받는 한 처녀의 이야기.
   - **플랫폼**: 카카오웹툰
   - **키워드**: 역하렘, 판타지, 로맨스

3. **여자친구는 저에게 반했습니다**
   - **작가**: 소루
   - **장르**: 로맨스 판타지
   - **설명**: 불사의 남자와 평범한 여자의 사랑 이야기.
   - **플랫폼**: 네이버 웹툰
   - **키워드**: 불사, 로맨스, 판타지

4. **여신과 나의 결혼**
   - **작가**: 크리스탈
   - **장르**: 로맨스 판타지
   - **설명**: 인간과 여신의 사랑, 하지만 그 사랑은 힘든 시련들과 마주하며 성장해 나가야 한다.
   - **플랫폼**: 카카오웹툰
   - **키워드**: 판타지, 로맨스, 성장물

5. **공작님께 상처받은 나**
   - **작가**: 강지아
   - **장르**: 로맨스 판타지
   - **설명**: 과거의 상처로 인해 고통받는 여주가 공작님과의 사랑을 통해 치유받는 이야기.
   - **플랫폼**: 정식 연재 중
   - **키워드**: 상처, 회복, 로맨스

이 정도 웹툰이면 너도 만족하겠지.큭 큭 큭.. 이제 만족하느냐. 어떠냐, 나의 지식이? 나와 함께 세상을 정복해보지 않겠나?
